In [1]:
import numpy as np

npzfile = np.load('../week10/CBCL.npz')
trainface = npzfile['arr_0']
trainnonface = npzfile['arr_1']
testface = npzfile['arr_2']
testnonface = npzfile['arr_3']

In [2]:
trnn = trainnonface.shape[0]
trpn = trainface.shape[0]
tepn = testface.shape[0]
tenn = testnonface.shape[0]


In [4]:
fn = 0
ftable = []
# 窮舉所有特徵
for y in range(19):
    for x in range(19):
        for h in range(2, 20):
            for w in range(2, 20):
                if y+h <= 19 and x+w*2 <= 19:
                    fn += 1
                    ftable.append([0, y, x, h, w])
print(fn)

for y in range(19):
    for x in range(19):
        for h in range(2, 20):
            for w in range(2, 20):
                if y+h <= 19 and x+w*2 <= 19:
                    fn += 1
                    ftable.append([1, y, x, h, w])

for y in range(19):
    for x in range(19):
        for h in range(2, 20):
            for w in range(2, 20):
                if y+h <= 19 and x+w*2 <= 19:
                    fn += 1
                    ftable.append([2, y, x, h, w])

for y in range(19):
    for x in range(19):
        for h in range(2, 20):
            for w in range(2, 20):
                if y+h <= 19 and x+w*2 <= 19:
                    fn += 1
                    ftable.append([3, y, x, h, w])

12312


In [5]:
def fe(sample, ftable, c):
    ftype = ftable[c][0]
    y = ftable[c][1]
    x = ftable[c][2]
    h = ftable[c][3]
    w = ftable[c][4]
    T = np.arange(361).reshape((19, 19))
    if ftype == 0:
        idx1 = T[y:y+h, x:x+w].flatten() 
        idx2 = T[y:y+h, x+w:x+w*2].flatten() 
        # 白色減黑色
        output = np.sum(sample[:, idx1], axis=1) - np.sum(sample[:, idx2], axis=1)
    elif ftype == 1:
        idx1 = T[y:y+h, x:x+w].flatten() 
        idx2 = T[y:y+h*2, x+w:x+w].flatten() 
        output = np.sum(sample[:, idx2], axis=1) - np.sum(sample[:, idx1], axis=1)
    elif ftype == 2:
        idx1 = T[y:y+h, x:x+w].flatten() 
        idx2 = T[y:y+h, x+w:x+w*2].flatten() 
        idx3 = T[y:y+h, x+w*2:x+w*3].flatten() 
        output = np.sum(sample[:, idx1], axis=1) - np.sum(sample[:, idx2], axis=1) + np.sum(sample[:, idx3], axis=1)
    elif ftype == 3:
        idx1 = T[y:y+h, x:x+w].flatten() 
        idx2 = T[y:y+h*2, x+w:x+w].flatten() 
        idx3 = T[y+h:y+h*2, x:x+w].flatten() 
        idx4 = T[y+h:y+h*2, x+w:x+w*2].flatten() 
        output = np.sum(sample[:, idx1], axis=1) - np.sum(sample[:, idx2], axis=1) + np.sum(sample[:, idx4], axis=1) -  np.sum(sample[:, idx3], axis=1)
        
    return output

In [7]:
%%time

trpf = np.zeros((trpn, fn))
trnf = np.zeros((trnn, fn))

for c in range(fn):
    trpf[:, c] = fe(trainface, ftable, c)
    trnf[:, c] = fe(trainnonface, ftable, c)

In [ ]:
def WC(pw, nw, pf, nf):
    maxf = max(pf.max(), nf.max())
    minf = min(pf.min(), nf.min())
    theta = (maxf-minf) / 10 + minf
    error = np.sum(pw[pf<theta]) + np.sum(nw[nf>=theta])
    polaity =1
    if error > 0.5:
        polaity = 0
        error = 1 - error
    min_theta = theta
    min_error = error
    min_polarity = polaity
    for i in range(2, 10):
        theta = (maxf - minf) * i / 10 + minf
        error = np.sum(pw[pf < theta]) + np.sum(nw[nf>=theta])
        polaity =1
        if error > 0.5:
            polaity = 0
            error = 1 - error
        if error < min_error:
            min_theta = theta
            min_error = error
            min_polarity = polaity
    return min_theta, min_error, min_polarity

In [ ]:
pw = np.ones((trpn, 1))/ trpn/ 2
nw = np.ones((trnn, 1))/ trnn/ 2

me, mt, mp = WC(pw, nw, trpf[:, 0], trnf[:, 0])

mine = me
mini = 0
for i in range(1, fn):
    me, mt, mp = WC(pw, nw, trpf[:, i], trnf[:, 1])
    if me < mine:
        mine = me
        mini = i
print([mini, mine])
